### Матричные факторизации

In [1]:
!pip install implicit lightfm faiss

     |████████████████████████████████| 1.1MB 22.2MB/s 
     |████████████████████████████████| 317kB 50.2MB/s 
     |████████████████████████████████| 4.7MB 51.6MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406427 sha256=5b40cf6d322ce5a9e5fe671a23c2e8a202ab40d3e1f3f8560b3b5ffd8b81ab99
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705340 sha256=16a3c7787d7397039ced7592c9f1f7f40b2f6d9ebbc3318b4ae082aa5a62f4d5
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built implicit lightfm


In [2]:
!apt-get install libopenblas-dev
!apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-dev is already the newest version (0.2.20+ds-4).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (421 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 160980 files and directories currently installed.)
Preparin

In [3]:
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip
!mv ml-1m/*.dat .
!rm -r ml-1m*

--2021-03-24 18:52:51--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  16.8MB/s    in 0.3s    

2021-03-24 18:52:51 (16.8 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [4]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

In [5]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [6]:
movie_info = pd.read_csv('movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

In [7]:
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [8]:
implicit_ratings = ratings[ratings.rating >= 4]

In [9]:
implicit_ratings.head()

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5


In [10]:
users = implicit_ratings.user_id
movies = implicit_ratings.movie_id
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [11]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

In [12]:
model.fit(user_item_t_csr)

In [13]:
movie_info.head()

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

In [15]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '1526    Hercules (1997)',
 '360    Lion King, The (1994)',
 '2252    Pleasantville (1998)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [16]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [17]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [18]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [19]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '957    African Queen, The (1951)',
 '1892    Rain Man (1988)',
 '1884    French Connection, The (1971)',
 '847    Godfather, The (1972)']

 Создадим модель, которая будет получать embeddings для пользователей и фильмов. Эта модель будет генерировать рекоммендации и similars с помощью библиотеки **faiss** (на gpu работает супер быстро)

In [20]:
from typing import List
import faiss

class RecommendModel:
    def __init__(self, user_embeddings: np.ndarray, item_embeddings: np.ndarray):
        assert user_embeddings.shape[1] == item_embeddings.shape[1]
        d = user_embeddings.shape[1]

        self.indices = {}
        self.embeddings = {}

        for name, embeddings in [
            ('user', user_embeddings),
            ('item', item_embeddings),
        ]:
            self.embeddings[name] = embeddings
            self.indices[name] = faiss.IndexFlatL2(d)
            self.indices[name].add(embeddings)

    def find_similars(self, name: str, i: int, top_k: int = 10) -> List[int]:
        embedding = self.embeddings[name][i].reshape((1, -1))
        d, i = self.indices[name].search(embedding, top_k)
        return zip(list(i[0]), list(d[0]))

    def similar_items(self, item_id: int):
        return self.find_similars('item', item_id, 10)


    def recommend(self, user_id: int, data: sp.csr.csr_matrix) -> List[int]:
        user_embedding = self.embeddings['user'][user_id].reshape((1, -1))

        dist, nearest = self.indices['item'].search(user_embedding, 100)

        row = data[user_id, :].toarray().flatten()
        recs = []
        for i, d in zip(list(nearest[0]), list(dist[0])):
            if row[i] == 0:
                recs.append((i, d))
        return recs[:10]

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [66]:
from scipy.sparse.linalg import spsolve
from scipy import sparse
from tqdm.auto import tqdm

In [ ]:
from numpy.random import default_rng
rng = default_rng()

def implicit_als(sparse_data, alpha=40, iterations=10, reg=0.1, hidden=10):
    confidence = sparse_data * alpha
    X = sparse.csr_matrix(
        rng.standard_normal((sparse_data.shape[0], hidden)) * 0.001
    )
    Y = sparse.csr_matrix(
        rng.standard_normal((sparse_data.shape[1], hidden)) * 0.001
    )
    
    reg_I = reg * sparse.eye(hidden)

    for i in tqdm(range(iterations)):
        yT_y = Y.T.dot(Y)
        xT_x = X.T.dot(X)

        def calc_opt_value(M, other, row):
          p = row.copy()
          p[p > 0] = 1

          diag_minus = sparse.diags(row, [0])
          diag = diag_minus + sparse.eye(M.shape[0])

          val1 = M.T.dot(diag_minus).dot(M)
          val2 = M.T.dot(diag).dot(p.T)
          return spsolve(other + val1, val2)

        for u in tqdm(range(sparse_data.shape[0]), leave=False):
            u_row = confidence[u, :].toarray() 
            X[u] = calc_opt_value(Y, yT_y, u_row)
    
        for i in tqdm(range(sparse_data.shape[1]), leave=False):
            i_row = confidence[:, i].T.toarray()
            Y[i] = calc_opt_value(X, xT_x, i_row)
    return X, Y

In [ ]:
user_vecs, item_vecs = implicit_als(user_item_csr, iterations=4, hidden=20, alpha=40)

In [ ]:
user_embeddings = user_vecs.toarray().astype(np.float32)
item_embeddings = item_vecs.toarray().astype(np.float32)

In [ ]:
als_model = RecommendModel(user_embeddings, item_embeddings)

In [ ]:
get_recommendations(4, als_model)

['1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '585    Terminator 2: Judgment Day (1991)',
 '1182    Aliens (1986)',
 '2502    Matrix, The (1999)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '847    Godfather, The (1972)',
 '453    Fugitive, The (1993)',
 '537    Blade Runner (1982)',
 '2847    Total Recall (1990)']

In [ ]:
get_similars(1, als_model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '1245    Groundhog Day (1993)',
 '2327    Shakespeare in Love (1998)',
 '2918    Who Framed Roger Rabbit? (1988)',
 '2928    Being John Malkovich (1999)',
 "2286    Bug's Life, A (1998)",
 '2647    Ghostbusters (1984)',
 '352    Forrest Gump (1994)',
 '1892    Rain Man (1988)']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

Напишем простой датасет и модельку используя pytorch. 
Будем подавать все позитивные примеры, и семплить к ним негативные.
Обучим небольшое количество эпох с помощью Adam. 

Попробуем переобучить модель на текущих данных, поэтому не будем использовать валидационный сабсет

In [124]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [125]:
from torch.utils.data import Dataset
from collections import defaultdict


class FilmDataset(Dataset):
    def __init__(self, sparse_data):
        self.n_users = sparse_data.shape[0]
        self.n_items = sparse_data.shape[1]

        self.pos_samples = defaultdict(list)

        users, movies = sparse_data.nonzero()

        self.pos_pairs = []
        for user, movie in zip(users, movies):
            self.pos_samples[user].append(movie)
            self.pos_pairs.append((user, movie))

    def __len__(self):
        return len(self.pos_pairs)

    def __getitem__(self, i):
        user, pos_sample = self.pos_pairs[i]
        neg_sample = np.random.randint(0, self.n_items)
        while neg_sample in self.pos_samples[user]:
            neg_sample = np.random.randint(0, self.n_items) 
        return user, pos_sample, neg_sample

In [126]:
dataset = FilmDataset(user_item)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True,
    num_workers=2,
    pin_memory=True,
)

In [127]:
class BPR(nn.Module):
    def __init__(self, hidden, reg, init_mean=0, init_stdev=0.01):
        super().__init__()

        self.dataset = dataset
        
        self.n_users = dataset.n_users
        self.n_items = dataset.n_items

        self.hidden = hidden

        self.reg = reg
        self.init_mean = init_mean
        self.init_stdev = init_stdev

        self.U = torch.normal(
            mean=self.init_mean * torch.ones(self.n_users, self.hidden),
            std = self.init_stdev
        ).requires_grad_()
        self.V = torch.normal(
            mean=self.init_mean * torch.ones(self.n_items, self.hidden),
            std = self.init_stdev
        ).requires_grad_()
        
    def forward(self, u, i, j):
        y_ui = torch.diag(torch.mm(self.U[u], self.V[i].t()))
        y_uj = torch.diag(torch.mm(self.U[u], self.V[j].t()))
        reg = self.reg * (torch.sum(self.U[u] ** 2) + torch.sum(self.V[i] ** 2) + torch.sum(self.V[j] ** 2))
        loss = reg - torch.sum(torch.log(torch.sigmoid(y_ui - y_uj)))
        return y_ui, y_uj, loss

In [134]:
HIDDEN = 64
REG = 0.01
LR = 0.0003
EPOCHS = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [135]:
model = BPR(hidden=HIDDEN, reg=REG).to(DEVICE)

In [136]:
optim = torch.optim.Adam([model.U, model.V], lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optim, 
    T_max=EPOCHS * len(dataloader)
)

In [137]:
for epoch in range(EPOCHS):
    train_loss = 0
    for user, pos, neg in tqdm(dataloader):
        optim.zero_grad()

        user = user.to(DEVICE).long()
        pos = pos.to(DEVICE).long()
        neg = neg.to(DEVICE).long()

        y_ui, y_uj, loss = model(user, pos, neg)
        train_loss += loss.item()

        loss.backward()
        optim.step()
        scheduler.step()
    
    train_loss /= len(dataloader)
    print('Epoch {}/{}: train_loss = {:.5f}, lr = {:.5f}'.format(
        epoch, EPOCHS, train_loss, scheduler.get_last_lr()[-1]
      ))



Epoch 0/10: train_loss = 30.07530, lr = 0.00029



Epoch 1/10: train_loss = 20.95573, lr = 0.00027



Epoch 2/10: train_loss = 20.38836, lr = 0.00024



Epoch 3/10: train_loss = 19.97722, lr = 0.00020



Epoch 4/10: train_loss = 19.69065, lr = 0.00015





Epoch 5/10: train_loss = 19.39651, lr = 0.00010



Epoch 6/10: train_loss = 19.26340, lr = 0.00006



Epoch 7/10: train_loss = 19.15235, lr = 0.00003



Epoch 8/10: train_loss = 19.08537, lr = 0.00001



Epoch 9/10: train_loss = 19.06426, lr = 0.00000


In [138]:
user_embeddings = model.U.detach().numpy()
item_embeddings = model.V.detach().numpy()

In [139]:
bpr_model = RecommendModel(user_embeddings, item_embeddings)

In [140]:
get_recommendations(4, bpr_model)

['2789    American Beauty (1999)',
 '589    Silence of the Lambs, The (1991)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '315    Shawshank Redemption, The (1994)',
 "523    Schindler's List (1993)",
 '2693    Sixth Sense, The (1999)',
 '1179    Princess Bride, The (1987)',
 '847    Godfather, The (1972)',
 '1250    Back to the Future (1985)',
 '1245    Groundhog Day (1993)']

In [141]:
get_similars(1, bpr_model)

['0    Toy Story (1995)',
 '453    Fugitive, The (1993)',
 '2647    Ghostbusters (1984)',
 '108    Braveheart (1995)',
 '1250    Back to the Future (1985)',
 '1179    Princess Bride, The (1987)',
 '1220    Terminator, The (1984)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '352    Forrest Gump (1994)',
 "2849    Ferris Bueller's Day Off (1986)"]

 С рекоммендациями модель более менее справилась, с поиском похожих не очень. Для улучшения результатов нужно подобрать гиперпараметры, сменить стратегию тренировки (как минимум обучать больше эпох). 

Также можно попробовать поменять лосс на arcface/cosface, они хорошо справляются с созданием пространства эмбеддингов.